# To run this notebook, install fastp, parallel, and kaiju and add them to your PATH
# make sure all fastq files are in working directory
fastp: 
wget http://opengene.org/fastp/fastp
chmod a+x ./fastp

parallel: 
wget http://ftpmirror.gnu.org/parallel/parallel-20150322.tar.bz2 
bzip2 -dc parallel-20150322.tar.bz2 | tar xvf - 
cd parallel-20150322
./configure && make && make install

kaiju: 
git clone https://github.com/bioinformatics-centre/kaiju.git

Also, create or install reference database from Kaiju. This project uses nr_euk and it is installed as follows: 
mkdir kaijudb
cd kaijudb
kaiju-makedb -s nr_euk

In [1]:
# Running fastp to clean sequences 
# the --dryrun will present each command as it will run without running it. Remove --dryrun once ready to run 
ls *fastq.gz | parallel --dryrun -j32 --max-args=2 fastp -h ~/[path_to_project]/{1.}.html  -i ~/[path_to_project]/{1} -I ~/[path_to_project]/{2} -o ./{1.}.gz -O ~/[path_to_project]/fastp_out/{2.}.gz

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1086926584.py, line 2)

In [ ]:
# Running Kaiju 
ls *qc.fastq.gz | parallel --dryrun -j15 --max-args=2 kaiju -t ~/[path_to_kaijudb]/nodes.dmp -f ~/[path_to_kaijudb]/kaiju_db_nr_euk.fmi -i {1} -j {2} -o ~/[path_to_project]/kaiju_out/{1.}.out

In [ ]:
# Running Kaiju2table 
ls *.out | parallel --dryrun -j15 kaiju2table -t ~/[path_to_kaijudb]/nodes.dmp -n ~/[path_to_kaijudb]/names.dmp -r species -l domain,superkingdom,phylum,class,order,family,genus,species -o {.}.tsv {}

In [ ]:
# Convert .tsv to OTU table 
python3 kaiju_table_to_OTU.py